## Segmenting and Clustering Neighborhoods in Toronto PART 1

    In this assignment we will be required to explore, segment, and cluster the neighborhoods in the city of Toronto.
    For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset(Tutorial from Week3 Capstone Project).

In [50]:
# import required libraries
import numpy as np
import pandas as pd
import html5lib 
import requests
import lxml
import html.parser

from bs4 import BeautifulSoup
import bs4
from pandas.io.html import read_html

### Read Wiki Tables from Wikipedia url using Pandas

In [9]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, attrs={"class":"wikitable"})
print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [13]:
wikitables[0]

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [15]:
type(wikitables[0])

pandas.core.frame.DataFrame

#### We have extracted a table of dataframe of pandas lets name it as toronto_with_notassigned because it has 'Not assigned' boroughs, lets clean the table and name it toronto

In [22]:
toronto_with_notassigned = wikitables[0]

In [23]:
toronto_with_notassigned.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [63]:
# size of dataframe before dropping Not assigned Boroughs
toronto_with_notassigned.shape

(180, 3)

In [60]:
toronto = toronto_with_notassigned.drop(toronto_with_notassigned[toronto_with_notassigned.Borough == 'Not assigned'].index)

In [61]:
toronto.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [64]:
# size of dataframe after dropping not assigned boroughs
toronto.shape

(103, 3)

In [67]:
toronto[toronto.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


There are 0 Not assigned neighborhoods. That is , none of the boroughs has its own Neighborhood as name